In [95]:
# import Dependecies
import pandas as pd
import gensim.downloader as api
from gensim.models import KeyedVectors,Word2Vec
import random
from nltk.tokenize import sent_tokenize

In [61]:
# Load Synonym File
synonym_file = pd.read_csv("synonym.csv")

In [62]:
# Intialize the Model Name
model_name_1 = "word2vec-google-news-300"

In [63]:
# Load the Data model
data_model_1 = api.load(model_name_1)

In [74]:
# Helper function
def detail_helper(data_model,file_name) :
    correct_answer = 0
    guess = 0
    Found = False
    with open(f'{file_name}.csv','w') as file:
        file.write("question,answer,guess,label\n")
        for iter,row in synonym_file.iterrows():
            question_word = row['question']
            answer_word = row['answer']
          
            if (question_word in data_model and any(row[f'{i}'] in data_model for i in range(0,4))):
                similarities = [(row[f'{i}'], data_model.similarity(question_word, row[f'{i}'])) for i in range(0,4) if row[f'{i}'] in data_model]
                closest,_ = max(similarities, key=lambda x: x[1])
                if (closest == answer_word):
                    file.write(f'{question_word},{answer_word},{closest},correct\n')
                    correct_answer+= 1
                    if(Found == False):
                        guess += 1
                else:
                    file.write(f'{question_word},{answer_word},{closest},wrong\n')
                    if(Found == False):
                        guess += 1      
            else:
                random_word = row[f'{random.randint(0,3)}']
                file.write(f'{question_word},{answer_word},{random_word},guess\n')
                Found = True
    
    with open('analysis.csv','a') as analysis:
        analysis.write(f'{file_name},{len(data_model)},{correct_answer},{guess},{correct_answer/guess}\n')
    
    return {'file_name' : file_name , 'length' : len(data_model),'correct_answer' : correct_answer, 'guess' : guess, 'accuracy' : correct_answer/guess }
     
        

In [76]:
# Call function to create analysis files
detail_helper(data_model_1,word2vec-google-news-300)

{'file_name': 'hello',
 'length': 3000000,
 'correct_answer': 70,
 'guess': 63,
 'accuracy': 1.1111111111111112}

In [67]:
# Task 2
# First Model -- Same Embeddings
model_name_2 = "glove-twitter-50"

In [68]:
# Load the model
data_model_2 = api.load(model_name_2)

In [75]:
# Call the Helper Function 
detail_helper(data_model_2,model_name_2)

{'file_name': 'glove-twitter-50',
 'length': 1193514,
 'correct_answer': 36,
 'guess': 36,
 'accuracy': 1.0}

In [43]:
# Second Model -- Same Emeddings
model_name_3 = "glove-wiki-gigaword-50"

In [44]:
# Load the model
data_model_3 = api.load(model_name_3)

[==================================================] 100.0% 66.0/66.0MB downloaded


In [45]:
# Call the Helper
detail_helper(data_model_3,model_name_3)

In [47]:
# Third Model -- Different Embedding same Corpus
model_name_4 = "glove-wiki-gigaword-200"

In [48]:
# Load the model
data_model_4 = api.load(model_name_4)

[==================================================] 100.0% 252.1/252.1MB downloaded


In [50]:
# Call Helper
detail_helper(data_model_4,model_name_4)

In [51]:
# Fourth Model -- Different Embedding same corpus
model_name_5 = "glove-wiki-gigaword-300"

In [52]:
# load the data
data_model_5 = api.load(model_name_5)

[==================================================] 100.0% 376.1/376.1MB downloaded


In [59]:
# Call the Helper
print(detail_helper(data_model_5,model_name_5))

{'file_name': 'glove-wiki-gigaword-300', 'length': 400000, 'correct_answer': 71, 'guess': 80, 'accuracy': 0.8875}


In [ ]:
# TODO Analysis Random Baseline -- Human Gold Standard

In [79]:
# Task 3
# Array of Books
book_array = ['./pg72232.txt','./pg72233.txt','./pg72234.txt','./pg72235.txt','./pg72236.txt']

In [91]:
def process_books(book_array):
    sentences = []
    for i in book_array:
        with open(i, 'r',encoding='utf-8') as file:
            sentences.append(sent_tokenize(file.read()))
    
    return sentences
            

In [98]:
# Define the constants
# Window Size Used
window_size = [5,10]
# Number of Embeddings
embeddings = [100,200]

In [107]:
# Train model with sentences
def train_model(sentence_array,window_size,embeddinsg) :
    model = Word2Vec(sentence_array,vector_size=embeddings,window=window_size)
    return model 

In [108]:
train_model_1 = train_model(process_books(book_array),window_size[0],embeddings[0])
detail_helper(train_model_1,f'own-corpus-E{embeddings[0]}-W{window_size[0]}')

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'